In [26]:
#load libraries
import pandas as pd
import json
import requests
from pandas.io.json import json_normalize

In [51]:
#load orders json file


with open('orders.json') as f:
    orders_json = json.load(f)

In [52]:
#flatten json data


orders_df = json_normalize(orders_json)

In [53]:
#add new date only column
orders_df['Date'] = pd.to_datetime(orders_df['created_at']).dt.date

In [54]:
#min and max date
max_date = orders_df['Date'].max()
min_date = orders_df['Date'].min()
print("min date is " + str(min_date) + " and max date is " + str(max_date))

min date is 2020-01-01 and max date is 2020-03-24


In [55]:
CURRENCY_US = "USD"
CURRENCY_CAD = "CAD"

In [57]:
#api url sample  - https://api.exchangeratesapi.io/history?start_at=2018-01-01&end_at=2018-09-01&symbols=ILS,JPY
#"&symbols=" + CURRENCY_US + "," + CURRENCY_CAD + ""
#GET https://api.exchangeratesapi.io/history?start_at=2018-01-01&end_at=2018-09-01&base=USD



url = "https://api.exchangeratesapi.io/history?start_at=" + str(min_date) + "&end_at=" + str(max_date) + "&base=" + CURRENCY_US + "&symbols=" + CURRENCY_CAD

exchange_information = []
response = requests.get(url)
if (response.status_code != 200):
    print("URL is not working")
    
else:
    response = requests.get(url)
    exchange_rates_data = response.json()
    exchange_rates_cad = exchange_rates_data["rates"]
    for exchange_rate_cad in exchange_rates_cad: 
        exchange_information.append([exchange_rate_cad,exchange_rates_cad[exchange_rate_cad]["CAD"]])
    exchange_df = pd.DataFrame(exchange_information)
    #add headers
    exchange_df.columns = ['Date','CAD_RATE']

In [58]:
exchange_df.sort_values(by=['Date'], inplace=True)
exchange_df

,Date,CAD_RATE
55,2020-01-02,1.299830
31,2020-01-03,1.298197
47,2020-01-06,1.297570
0,2020-01-07,1.299678
43,2020-01-08,1.301844
19,2020-01-09,1.305581
7,2020-01-10,1.307186
41,2020-01-13,1.304512
6,2020-01-14,1.307063
42,2020-01-15,1.307216


In [60]:
orders_df['Date'] = orders_df['Date'].astype(str)

orders_df.reset_index(drop=True, inplace=True)
exchange_df.reset_index(drop=True, inplace=True)

#merge the exchange information
orders_CAD_df = pd.merge(orders_df, exchange_df, how='left',on = 'Date')

In [61]:
orders_CAD_df

,id,total_price,created_at,line_items,customer.id,customer.name,customer.email,Date,CAD_RATE
0,8369263756632563,79.39,2020-03-07T14:31:11Z,"[{'id': 610448, 'product_id': 632910392, 'prod...",47178,William Doe,william.doe@gmail.com,2020-03-07,NaN
1,7262170348080494,86.98,2020-03-01T09:16:30Z,"[{'id': 997208, 'product_id': 278266679, 'prod...",94720,Emile Tumson,emile.tumson@gmail.com,2020-03-01,NaN
2,7722055557038194,36.99,2020-01-01T09:17:03Z,"[{'id': 759658, 'product_id': 573794972, 'prod...",59933,Ethan Jones,ethan.jones@gmail.com,2020-01-01,NaN
3,4280852170163518,55.98,2020-03-10T11:59:46Z,"[{'id': 317808, 'product_id': 477365777, 'prod...",23955,Daniel Smith,daniel.smith@gmail.com,2020-03-10,1.367779
4,5058732129957127,309.98,2020-03-15T07:27:24Z,"[{'id': 604665, 'product_id': 510650809, 'prod...",59933,Ethan Jones,ethan.jones@gmail.com,2020-03-15,NaN
5,6431399815998774,315.97,2020-03-12T07:16:07Z,"[{'id': 997208, 'product_id': 278266679, 'prod...",23955,Daniel Smith,daniel.smith@gmail.com,2020-03-12,1.381139
6,5791957346866373,36.99,2020-03-24T09:52:40Z,"[{'id': 759658, 'product_id': 573794972, 'prod...",47178,William Doe,william.doe@gmail.com,2020-03-24,1.449414
7,2079640804901496,55.98,2020-02-19T19:33:57Z,"[{'id': 317808, 'product_id': 477365777, 'prod...",23955,Daniel Smith,daniel.smith@gmail.com,2020-02-19,1.322315
8,2249838395290890,309.98,2020-03-13T12:15:51Z,"[{'id': 604665, 'product_id': 510650809, 'prod...",47178,William Doe,william.doe@gmail.com,2020-03-13,1.385897
9,7638742701884072,35.39,2020-03-08T05:28:01Z,"[{'id': 317808, 'product_id': 477365777, 'prod...",59933,Ethan Jones,ethan.jones@gmail.com,2020-03-08,NaN


In [81]:
#exchange_df

#Update exchange information since some days are missing
exchange_update = []
daterange = pd.date_range(min_date, max_date)
for single_date in daterange:
    exchange_update.append(single_date.strftime("%Y-%m-%d"))
    exchange_update_df = pd.DataFrame(exchange_update)
    exchange_update_df.columns = ['Date'] #add headers

exchange_update_df = pd.merge(exchange_update_df, exchange_df, how='left',on = 'Date')
#exchange_update_df.fillna(method='ffill')   
#print(exchange_update_df)   



#for i, row in exchange_update_df.iterrows():
    print (row['Date'], row['CAD_RATE'])
    if row(row['CAD_RATE'])
#     i1, row1 = next(df.iterrows())
#     print row1['value']       

True

In [ ]:
#perform conversion from USD to CAD
orders_df_CAD['']

In [ ]:
#generate output


#refactor code